# 이수안컴퓨터연구소 머신러닝 #5
#### K-Nearest Neighbor Classifier
1. Iris
2. Breast Cancer
3. Wine
#### K-Nearest Neighbor Resressor
1. Boston
2. California

In [1]:
from sklearn import set_config
set_config(display='text')

In [2]:
import pandas as pd
import numpy as np
import multiprocessing
import matplotlib.pyplot as plt
plt.style.use(['seaborn-whitegrid'])

C:\Users\John\AppData\Local\Temp\ipykernel_12688\2441446269.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use(['seaborn-whitegrid'])


In [3]:
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.manifold import TSNE
from sklearn.datasets import load_iris, load_boston, load_breast_cancer, fetch_california_housing, load_wine
from sklearn.model_selection import train_test_split, cross_validate, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline, Pipeline

## K 최근접 이웃 분류

### 붓꽃 데이터

In [4]:
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['Target'] = iris.target
iris_df.head(2)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),Target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0


In [5]:
X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [6]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

In [7]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)
model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [8]:
print(f'train    data score : {model.score(X_train, y_train)}')
print(f'validate data score : {model.score(X_test, y_test)}')

train    data score : 1.0
validate data score : 0.9


In [9]:
model = KNeighborsClassifier()
model.fit(X_train_scale, y_train)
model.get_params()

{'algorithm': 'auto',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 5,
 'p': 2,
 'weights': 'uniform'}

In [10]:
print(f'train    data score : {model.score(X_train_scale, y_train)}')
print(f'validate data score : {model.score(X_test_scale, y_test)}')

train    data score : 0.975
validate data score : 0.9


In [11]:
cross_validate(estimator=KNeighborsClassifier(), X=X, y=y, cv=5,
               n_jobs=multiprocessing.cpu_count(), verbose=True)

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   5 out of   5 | elapsed:    0.7s finished


{'fit_time': array([0.        , 0.00100064, 0.00100064, 0.00100064, 0.        ]),
 'score_time': array([0.00200081, 0.00100017, 0.00100017, 0.00100017, 0.00150394]),
 'test_score': array([0.96666667, 1.        , 0.93333333, 0.96666667, 1.        ])}

In [12]:
param_grid = [{'n_neighbors': [3, 5, 7], 'weights': ['uniform','distance'],
               'algorithm': ['ball_tree', 'kd_tree', 'brute']}]

In [13]:
gs = GridSearchCV(estimator=KNeighborsClassifier(), param_grid=param_grid,
                  n_jobs=multiprocessing.cpu_count(), verbose=True)
                  
gs.fit(X, y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=KNeighborsClassifier(), n_jobs=24,
             param_grid=[{'algorithm': ['ball_tree', 'kd_tree', 'brute'],
                          'n_neighbors': [3, 5, 7],
                          'weights': ['uniform', 'distance']}],
             verbose=True)

In [14]:
gs.best_estimator_.get_params()

{'algorithm': 'ball_tree',
 'leaf_size': 30,
 'metric': 'minkowski',
 'metric_params': None,
 'n_jobs': None,
 'n_neighbors': 7,
 'p': 2,
 'weights': 'uniform'}

In [15]:
print(f'GridSearchCV best score: {gs.best_score_}')

GridSearchCV best score: 0.9800000000000001


# 시각화는 생략

### 차원 축소

In [16]:
tsne = TSNE(n_components=2)
X_comp = tsne.fit_transform(X)

c:\Users\John\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\John\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [17]:
iris_comp_df = pd.DataFrame(data=X_comp)
iris_comp_df['Target'] = y
iris_comp_df.head()

,0,1,Target
0,-4.252351,-25.039560,0
1,-3.113149,-27.409437,0
2,-4.122969,-27.821777,0
3,-3.846325,-28.063543,0
4,-3.882918,-24.939268,0


### 유방암 데이터

In [18]:
cancer = load_breast_cancer()
cancer_df = pd.DataFrame(cancer.data, columns=cancer.feature_names)
cancer_df['Target'] = cancer.target
cancer_df.head(2)

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,Target
0,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0
1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0


In [19]:
X, y = cancer.data, cancer.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [20]:
cancer_train_df = pd.DataFrame(data=X_train, columns=cancer.feature_names)

In [21]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

In [22]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)

KNeighborsClassifier()

In [23]:
print(f'train    data score : {model.score(X_train, y_train)}')
print(f'validate data score : {model.score(X_test, y_test)}')

train    data score : 0.945054945054945
validate data score : 0.9122807017543859


In [24]:
model = KNeighborsClassifier()
model.fit(X_train_scale, y_train)

KNeighborsClassifier()

In [25]:
print(f'train    data score : {model.score(X_train_scale, y_train)}')
print(f'validate data score : {model.score(X_test_scale, y_test)}')

train    data score : 0.9714285714285714
validate data score : 0.9736842105263158


In [26]:
estimator = make_pipeline(StandardScaler(), KNeighborsClassifier(),)

In [27]:
cross_validate(estimator=estimator, X=X, y=y, cv=5,
               n_jobs=multiprocessing.cpu_count(), verbose=True)

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   5 out of   5 | elapsed:    0.0s finished


{'fit_time': array([0.00100017, 0.00100017, 0.        , 0.0010004 , 0.0010004 ]),
 'score_time': array([0.00300074, 0.00300074, 0.00300074, 0.00200033, 0.00300121]),
 'test_score': array([0.96491228, 0.95614035, 0.98245614, 0.95614035, 0.96460177])}

In [28]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ('model', KNeighborsClassifier())])

In [29]:
param_grid = [{'model__n_neighbors': [3, 5, 7],
               'model__weights': ['uniform', 'distance'],
               'model__algorithm': ['ball_tree', 'kd_tree', 'brute']}]

In [30]:
gs = GridSearchCV(estimator=pipe, param_grid=param_grid,
                  n_jobs=multiprocessing.cpu_count(), verbose=True)

In [31]:
gs.fit(X, y)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', KNeighborsClassifier())]),
             n_jobs=24,
             param_grid=[{'model__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                          'model__n_neighbors': [3, 5, 7],
                          'model__weights': ['uniform', 'distance']}],
             verbose=True)

In [32]:
gs.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 KNeighborsClassifier(algorithm='ball_tree', n_neighbors=7))])

In [33]:
print(f'GridSearchCV data score : {gs.best_score_}')
# print(f'validate data score : {model.score(X_test, y_test)}')

GridSearchCV data score : 0.9701288619779538


In [34]:
tsne = TSNE(n_components=2)
X_comp = tsne.fit_transform(X)

c:\Users\John\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\John\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [35]:
cancer_comp_df = pd.DataFrame(data=X_comp)
cancer_comp_df['Target'] = y
cancer_comp_df.head(2)

,0,1,Target
0,-21.026014,25.455381,0
1,-23.731949,25.501947,0


In [36]:
model = KNeighborsClassifier()
model.fit(X_comp, y)
predict = model.predict(X_comp)

### 와인 데이터

In [37]:
wine = load_wine()
wine_df = pd.DataFrame(wine.data, columns=wine.feature_names)
wine_df['Target'] = wine.target
wine_df.head(2)

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,Target
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,0


In [38]:
X, y = wine.data, wine.target
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [39]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

In [40]:
model = KNeighborsClassifier()
model.fit(X_train, y_train)

KNeighborsClassifier()

In [41]:
print(f'train    data score : {model.score(X_train, y_train)}')
print(f'validate data score : {model.score(X_test, y_test)}')

train    data score : 0.8098591549295775
validate data score : 0.5833333333333334


In [42]:
model = KNeighborsClassifier()
model.fit(X_train_scale, y_train)

KNeighborsClassifier()

In [43]:
print(f'train    data score : {model.score(X_train_scale, y_train)}')
print(f'validate data score : {model.score(X_test_scale, y_test)}')

train    data score : 0.971830985915493
validate data score : 1.0


In [44]:
estimator = make_pipeline(StandardScaler(), KNeighborsClassifier(),)

In [45]:
cross_validate(estimator=estimator, X=X_train_scale, y=y_train, cv=5,
               n_jobs=multiprocessing.cpu_count(), verbose=True)

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   5 out of   5 | elapsed:    0.0s finished


{'fit_time': array([0.00100207, 0.00100207, 0.00100207, 0.        , 0.0010004 ]),
 'score_time': array([0.0010004 , 0.0010004 , 0.0010004 , 0.0010004 , 0.00099993]),
 'test_score': array([0.96551724, 0.93103448, 0.92857143, 0.92857143, 0.96428571])}

In [46]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ('model', KNeighborsClassifier())])

In [47]:
param_grid = [{'model__n_neighbors': [3, 5, 7],
               'model__weights': ['uniform', 'distance'],
               'model__algorithm': ['ball_tree', 'kd_tree', 'brute']}]

In [48]:
gs = GridSearchCV(estimator=pipe, param_grid=param_grid,
                  n_jobs=multiprocessing.cpu_count(), verbose=True)

In [49]:
# gs.fit(X, y)
# gs.fit(X_train, y_train)
gs.fit(X_train_scale, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', KNeighborsClassifier())]),
             n_jobs=24,
             param_grid=[{'model__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                          'model__n_neighbors': [3, 5, 7],
                          'model__weights': ['uniform', 'distance']}],
             verbose=True)

In [50]:
gs.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 KNeighborsClassifier(algorithm='ball_tree', n_neighbors=7))])

In [51]:
print(f'GridSearchCV data score : {gs.best_score_}')
print(f'validate data score : {model.score(X_test_scale, y_test)}')

GridSearchCV data score : 0.9504926108374384
validate data score : 1.0


In [52]:
tsne = TSNE(n_components=2)
X_comp = tsne.fit_transform(X)

c:\Users\John\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_t_sne.py:800: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
c:\Users\John\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\manifold\_t_sne.py:810: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


In [53]:
cancer_comp_df = pd.DataFrame(data=X_comp)
cancer_comp_df['Target'] = y
cancer_comp_df.head(2)

,0,1,Target
0,-10.501795,5.287938,0
1,-10.641080,4.515627,0


In [54]:
model = KNeighborsClassifier()
model.fit(X_comp, y)
predict = model.predict(X_comp)

# 최근접 이웃 회귀
### 보스턴 주택 가격 데이터

In [55]:
import warnings
warnings.filterwarnings('ignore')

In [56]:
boston = load_boston()
boston_df = pd.DataFrame(boston.data,  columns=boston.feature_names)
boston_df['Target'] = boston.target
X, y = load_boston(return_X_y=True)

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [58]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

In [59]:
model = KNeighborsRegressor()
model.fit(X_train, y_train)

KNeighborsRegressor()

In [60]:
print(f'train    data score : {model.score(X_train, y_train)}')
print(f'validate data score : {model.score(X_test, y_test)}')

train    data score : 0.7277183004091167
validate data score : 0.4655311917789289


In [61]:
model = KNeighborsRegressor()
model.fit(X_train_scale, y_train)

KNeighborsRegressor()

In [62]:
print(f'train    data score : {model.score(X_train_scale, y_train)}')
print(f'validate data score : {model.score(X_test_scale, y_test)}')

train    data score : 0.8250180154784053
validate data score : 0.8133035230898318


In [63]:
estimator = make_pipeline(StandardScaler(), KNeighborsRegressor(),)

In [64]:
cross_validate(estimator=estimator, X=X_train_scale, y=y_train, cv=5,
               n_jobs=multiprocessing.cpu_count(), verbose=True)

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   5 out of   5 | elapsed:    0.0s finished


{'fit_time': array([0.00099993, 0.0010004 , 0.0010004 , 0.0010004 , 0.        ]),
 'score_time': array([0.0010004 , 0.        , 0.        , 0.        , 0.00150371]),
 'test_score': array([0.7014337 , 0.54014921, 0.72078452, 0.75266249, 0.72513839])}

In [65]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ('model', KNeighborsRegressor())])

In [66]:
param_grid = [{'model__n_neighbors': [3, 5, 7],
               'model__weights': ['uniform', 'distance'],
               'model__algorithm': ['ball_tree', 'kd_tree', 'brute']}]

In [67]:
gs = GridSearchCV(estimator=pipe, param_grid=param_grid,
                  n_jobs=multiprocessing.cpu_count(), verbose=True)

In [68]:
# gs.fit(X, y)
# gs.fit(X_train, y_train)
gs.fit(X_train_scale, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', KNeighborsRegressor())]),
             n_jobs=24,
             param_grid=[{'model__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                          'model__n_neighbors': [3, 5, 7],
                          'model__weights': ['uniform', 'distance']}],
             verbose=True)

In [69]:
gs.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 KNeighborsRegressor(algorithm='ball_tree',
                                     weights='distance'))])

In [70]:
print(f'GridSearchCV data score : {gs.best_score_}')
print(f'validate data score : {model.score(X_test_scale, y_test)}')

GridSearchCV data score : 0.7352349481098936
validate data score : 0.8133035230898318


In [71]:
tsne = TSNE(n_components=1)
X_comp = tsne.fit_transform(X)

In [72]:
boston_comp_df = pd.DataFrame(data=X_comp)
boston_comp_df['Target'] = y
boston_comp_df.head(2)

,0,Target
0,23.387152,24.0
1,38.532837,21.6


In [73]:
model = KNeighborsRegressor()
model.fit(X_comp, y)
predict = model.predict(X_comp)

### 캘리포니아 주택가격

In [74]:
califonia = fetch_california_housing()
califonia_df = pd.DataFrame(califonia.data,  columns=califonia.feature_names)
califonia_df['Target'] = califonia.target
X, y = fetch_california_housing(return_X_y=True)

In [75]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [76]:
scaler = StandardScaler()
X_train_scale = scaler.fit_transform(X_train)
X_test_scale = scaler.transform(X_test)

In [77]:
model = KNeighborsRegressor()
model.fit(X_train, y_train)

KNeighborsRegressor()

In [78]:
print(f'train    data score : {model.score(X_train, y_train)}')
print(f'validate data score : {model.score(X_test, y_test)}')

train    data score : 0.4490800936940853
validate data score : 0.17741312423575695


In [79]:
model = KNeighborsRegressor()
model.fit(X_train_scale, y_train)

KNeighborsRegressor()

In [80]:
print(f'train    data score : {model.score(X_train_scale, y_train)}')
print(f'validate data score : {model.score(X_test_scale, y_test)}')

train    data score : 0.7932714273452368
validate data score : 0.6895022035347143


In [81]:
estimator = make_pipeline(StandardScaler(), KNeighborsRegressor(),)

In [82]:
cross_validate(estimator=estimator, X=X_train_scale, y=y_train, cv=5,
               n_jobs=multiprocessing.cpu_count(), verbose=True)

[Parallel(n_jobs=24)]: Using backend LokyBackend with 24 concurrent workers.
[Parallel(n_jobs=24)]: Done   5 out of   5 | elapsed:    0.1s finished


{'fit_time': array([0.01251793, 0.01251793, 0.01251793, 0.01251793, 0.01251793]),
 'score_time': array([0.13820815, 0.14572668, 0.14471292, 0.14320922, 0.13820815]),
 'test_score': array([0.6774598 , 0.70123004, 0.69464494, 0.65303235, 0.66377247])}

In [83]:
pipe = Pipeline([('scaler', StandardScaler()),
                 ('model', KNeighborsRegressor())])

In [84]:
param_grid = [{'model__n_neighbors': [3, 5, 7],
               'model__weights': ['uniform', 'distance'],
               'model__algorithm': ['ball_tree', 'kd_tree', 'brute']}]

In [85]:
gs = GridSearchCV(estimator=pipe, param_grid=param_grid,
                  n_jobs=multiprocessing.cpu_count(), verbose=True)

In [86]:
# gs.fit(X, y)
# gs.fit(X_train, y_train)
gs.fit(X_train_scale, y_train)

Fitting 5 folds for each of 18 candidates, totalling 90 fits


GridSearchCV(estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                       ('model', KNeighborsRegressor())]),
             n_jobs=24,
             param_grid=[{'model__algorithm': ['ball_tree', 'kd_tree', 'brute'],
                          'model__n_neighbors': [3, 5, 7],
                          'model__weights': ['uniform', 'distance']}],
             verbose=True)

In [87]:
gs.best_estimator_

Pipeline(steps=[('scaler', StandardScaler()),
                ('model',
                 KNeighborsRegressor(algorithm='ball_tree', n_neighbors=7,
                                     weights='distance'))])

In [88]:
print(f'GridSearchCV data score : {gs.best_score_}')
print(f'validate data score : {model.score(X_test_scale, y_test)}')

GridSearchCV data score : 0.6884575319794031
validate data score : 0.6895022035347143


In [89]:
tsne = TSNE(n_components=1)
X_comp = tsne.fit_transform(X)

In [90]:
boston_comp_df = pd.DataFrame(data=X_comp)
boston_comp_df['Target'] = y
boston_comp_df.head(2)

,0,Target
0,55.102840,4.526
1,-68.376083,3.585


In [91]:
model = KNeighborsRegressor()
model.fit(X_comp, y)
predict = model.predict(X_comp)